# 第二章 例2-3-1

In [5]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Alias, Problem, Options
import pandas as pd

### Container

In [6]:
m = Container()

### Sets

In [7]:
i = Set(container=m, name="i", description="input", records=["sec1","sec2"])
j = Alias(m, name="j", alias_with=i)
i.name, j.name

('i', 'j')

### Data

In [8]:
data = pd.read_excel("chap2.xlsx", index_col=0)
data

FileNotFoundError: [Errno 2] No such file or directory: 'chap2.xlsx'

In [ ]:
use_data = data.loc[["sec1","sec2"],"use"].reset_index()
lab_data = data.loc["lab",["sec1","sec2"]].reset_index()
int_data = data.loc[["sec1","sec2"],["sec1","sec2"]].stack().reset_index()
Q0_data = data.loc[["sec1","sec2"],"total"].reset_index()
a_data = data.loc[["sec1","sec2"],["sec1","sec2"]].values / data.loc[["sec1","sec2"],"total"].values
a_data = pd.DataFrame(a_data, index=["sec1","sec2"], columns=["sec1","sec2"]).stack().reset_index()


### Parameters

In [ ]:
use = Parameter(
    container=m,
    name="use",
    domain=i,
    description="final use",
    records=use_data
)
print(use.records)


  index  value
0  sec1  100.0
1  sec2  530.0


In [ ]:
lab = Parameter(
    container=m,
    name="lab",
    domain=j,
    description="factor labor input",
    records=lab_data
)
print(lab.records)


  index  value
0  sec1  250.0
1  sec2  380.0


In [ ]:
int = Parameter(
    container=m,
    name="int",
    domain=[i, j],
    description="intermidiate input",
    records=int_data
)
print(int.records)


  level_0 level_1  value
0    sec1    sec1  200.0
1    sec1    sec2  300.0
2    sec2    sec1  150.0
3    sec2    sec2  320.0


In [ ]:
a = Parameter(
    container=m,
    name="a",
    domain=[i, j],
    description="direct input-output coefficients",
    records=a_data
)
print(a.records)


  level_0 level_1     value
0    sec1    sec1  0.333333
1    sec1    sec2  0.300000
2    sec2    sec1  0.250000
3    sec2    sec2  0.320000


In [ ]:
Q0 = Parameter(
    container=m,
    name="Q0",
    domain=i,
    description="initial value for total output",
    records=Q0_data
)
print(Q0.records)

  index   value
0  sec1   600.0
1  sec2  1000.0


### Variables

In [ ]:
Q = Variable(
    container=m,
    name="Q",
    domain=[i],
    type="Positive",
    description="total output",
)

obj = Variable(m, name="obj")

### Equations

In [ ]:
commodityequi = Equation( 
    container=m, name="commodityequi", domain=[i], description="commodity equality"  
) 
commodityequi[i] = Q[i] == Sum(j, a[i, j]* Q[j]) + use[i]  

dummy = Equation(m, "dummy")
dummy[...] = obj == 0


### Model

In [ ]:
Q1 = Parameter(
    container=m,
    name="Q1",
    domain=i,
    records=[["sec1",1],["sec2",2]]
)

IOmodel = Model(
    m,
    name="IOmodel",
    problem=Problem.LP,
    equations=m.getEquations(),
    sense="min",
    objective=obj
)
Q.l[i] = Q1[i]  

### Solve


In [ ]:
IOmodel.solve()
Q.records

,i,level,marginal,lower,upper,scale
0,sec1,600.0,0.0,0.0,inf,1.0
1,sec2,1000.0,0.0,0.0,inf,1.0


In [ ]:
model1 = Model(
    m,
    name="model1",
    problem="mcp",
    matches={commodityequi:Q}
)
Q.l[i] = Q1[i]  
model1.solve()
Q.records

,i,level,marginal,lower,upper,scale
0,sec1,600.0,7.105427e-14,0.0,inf,1.0
1,sec2,1000.0,-1.136868e-13,0.0,inf,1.0


In [ ]:
import sys
IOmodel.solve(output=sys.stdout, options=Options(equation_listing_limit=10, variable_listing_limit=10))

--- Job _aad8ece7-a351-46f7-8e6f-e7f37f745a22.gms Start 03/20/25 13:52:58 49.2.0 9b2b0844 DAX-DAC arm 64bit/macOS
--- Applying:
    /opt/anaconda3/lib/python3.12/site-packages/gamspy_base/gmsprmun.txt
--- GAMS Parameters defined
    LP CPLEX
    Input /var/folders/_3/k3k66fl17155q0_plbg58kn00000gn/T/tmpcq0x9y8m/_aad8ece7-a351-46f7-8e6f-e7f37f745a22.gms
    Output /var/folders/_3/k3k66fl17155q0_plbg58kn00000gn/T/tmpcq0x9y8m/_aad8ece7-a351-46f7-8e6f-e7f37f745a22.lst
    ScrDir /var/folders/_3/k3k66fl17155q0_plbg58kn00000gn/T/tmpcq0x9y8m/tmpmm_78xke/
    SysDir /opt/anaconda3/lib/python3.12/site-packages/gamspy_base/
    LogOption 3
    Trace /var/folders/_3/k3k66fl17155q0_plbg58kn00000gn/T/tmpcq0x9y8m/_aad8ece7-a351-46f7-8e6f-e7f37f745a22.txt
    License /opt/anaconda3/lib/python3.12/site-packages/gamspy_base/gamslice.txt
    OptDir /var/folders/_3/k3k66fl17155q0_plbg58kn00000gn/T/tmpcq0x9y8m/
    LimRow 10
    LimCol 10
    TraceOpt 3
    GDX /var/folders/_3/k3k66fl17155q0_plbg58kn00000

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,0,3,3,LP,CPLEX,0.001


In [ ]:
print(IOmodel.getEquationListing())

commodityequi(sec1)..  0.666666666666667*Q(sec1) - 0.3*Q(sec2) =E= 100 ; (LHS = 100)
commodityequi(sec2)..  - 0.25*Q(sec1) + 0.68*Q(sec2) =E= 530 ; (LHS = 530)
dummy..  obj =E= 0 ; (LHS = 0)


In [ ]:
model1.solve(output=sys.stdout, options=Options(equation_listing_limit=10, variable_listing_limit=10))


--- _aad8ece7-a351-46f7-8e6f-e7f37f745a22.gms(310) 4 Mb
--- Job _aad8ece7-a351-46f7-8e6f-e7f37f745a22.gms Start 03/20/25 13:52:58 49.2.0 9b2b0844 DAX-DAC arm 64bit/macOS
--- Applying:
    /opt/anaconda3/lib/python3.12/site-packages/gamspy_base/gmsprmun.txt
--- GAMS Parameters defined
    MCP PATH
    Input /var/folders/_3/k3k66fl17155q0_plbg58kn00000gn/T/tmpcq0x9y8m/_aad8ece7-a351-46f7-8e6f-e7f37f745a22.gms
    Output /var/folders/_3/k3k66fl17155q0_plbg58kn00000gn/T/tmpcq0x9y8m/_aad8ece7-a351-46f7-8e6f-e7f37f745a22.lst
    ScrDir /var/folders/_3/k3k66fl17155q0_plbg58kn00000gn/T/tmpcq0x9y8m/tmpmm_78xke/
    SysDir /opt/anaconda3/lib/python3.12/site-packages/gamspy_base/
    LogOption 3
    Trace /var/folders/_3/k3k66fl17155q0_plbg58kn00000gn/T/tmpcq0x9y8m/_aad8ece7-a351-46f7-8e6f-e7f37f745a22.txt
    License /opt/anaconda3/lib/python3.12/site-packages/gamspy_base/gamslice.txt
    OptDir /var/folders/_3/k3k66fl17155q0_plbg58kn00000gn/T/tmpcq0x9y8m/
    LimRow 10
    LimCol 10
    TraceOp

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,1.13686837721616E-13,2,2,MCP,PATH,0.004


In [ ]:
print(model1.getEquationListing())